# 使用postgres作为时序数据库

## 先构建一个表

create table monitor (
    time timestamp,
    tags_id integer,
    server_ip varchar(15),
    cpu integer,
    memory integer
);

## 插入数据

insert into monitor
select
    generate_series(now(), now() + '1 year', '1 second') as time,
    (random() * (10))::integer as tags_id,
    '192.168.1.' || (random() * (100))::integer::varchar as server_ip,
    (random() * (10))::integer as cpu,
    (random() * (10))::integer as memory;

## 查看表大小

select pg_size_pretty(pg_table_size('monitor'));

## 创建一个索引

create index monitor_ip_time_idx on monitor using btree (server_ip, time desc);

select *
from monitor
where time between '2024-10-19 00:00:00'::timestamp and '2024-10-20 00:00:00'::timestamp
and server_ip = '192.168.1.10';

explain analyse 
select *
from monitor
where time between '2024-10-19 00:00:00'::timestamp and '2024-10-20 00:00:00'::timestamp
and server_ip = '192.168.1.10';

## 查看索引大小

select pg_size_pretty(pg_relation_size('monitor_ip_time_idx'));

创建一个Brin索引

create index monitor_time__brin_idx on monitor using brin (time);

select *
from monitor
where time between '2024-10-19 00:00:00'::timestamp and '2024-10-20 00:00:00'::timestamp;

explain analyse
select *
from monitor
where time between '2024-10-19 00:00:00'::timestamp and '2024-10-20 00:00:00'::timestamp;

## 查看brin索引大小

select pg_size_pretty(pg_relation_size('monitor_time__brin_idx'));

## 使用分区表来提高大数据下的性能

create table monitor2 (
    time timestamp,
    tags_id integer,
    server_ip varchar(15),
    cpu integer,
    memory integer
) partition by range (time);

create table monitor2_2024_10
partition of monitor2
for values from ('2024-10-01 00:00:00+00') to ('2024-11-01 00:00:00+00');

create table monitor2_2024_11
partition of monitor2
for values from ('2024-11-01 00:00:00+00') to ('2024-12-01 00:00:00+00');

create index monitor2_time__brin_idx on monitor2 using brin (time);

insert into monitor2
select
    generate_series(now(), now() + '1 month', '1 second') as time,
    (random() * (10))::integer as tags_id,
    '192.168.1.' || (random() * (100))::integer::varchar as server_ip,
    (random() * (10))::integer as cpu,
    (random() * (10))::integer as memory;

select *
from monitor2
where time between '2024-10-19 00:00:00'::timestamp and '2024-10-20 00:00:00'::timestamp;

explain analyse 
select *
from monitor2
where time between '2024-10-19 00:00:00'::timestamp and '2024-10-20 00:00:00'::timestamp;